<a href="https://colab.research.google.com/github/sherryfengshopify/ai-gym/blob/main/Research_agent_langgraph_arxiv_serpapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Build a research agent prototype

In [2]:
# @title Extracting data from arxiv into a pandas dataframe and saing it as JSON

!pip install openai langchain langchain-community langchain-openai langgraph python-dotenv pypdf semantic_router pinecone serpapi google-search-results

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=88e55a8ef46787111da482a8c962a17a39fd0f8a14c6fd37cd15580824d4783b
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2c

In [ ]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET

# Namespace for ArXiv's Atom-based XML format.
ARXIV_NAMESPACE = '{http://www.w3.org/2005/Atom}'

def extract_from_arxiv(search_query='cat:cs.AI', max_results=100, json_file_path='files/arxiv_dataset.json'):
  """
  Fetches papers from the ArXiv API based on a search query, saves them as JSON,
  and returns a pandas DataFrame.

  Args:
      search_query (str): The search query to use in the API. Defaults to 'cat:cs.AI'.
      max_results (int): The maximum number of results to fetch. Defaults to 100.
      json_file_path (str): The path to save the JSON data to. Defaults to 'files/arxiv_dataset.json'.

  Returns:
      pd.DataFrame: A DataFrame containing the extracted paper information.
  """

  # Construct the URL for the API request.
  url = f'http://export.arxiv.org/api/query?search_query={search_query}&max_results={max_results}'

  # Send a GET request to the ArXiv API.
  response = requests.get(url)

  # Parse the XML response.
  root = ET.fromstring(response.content)

  papers = []

  # Loop through each "entry" in the XML, representing a single paper.